Hocam öncelikle selamlar
Colab linkinde resimlerin değişimiyle ilgili görüşlerimi yorum olarak ekledim. Burada da sizin halihazırda verdiğiniz kodlara yaptığım geliştirmeleri Markdown olarak alt satırlarına ekledim.
Jupyter windows kurulum için öncelikle cihazda python ve pip(kurumsal şirketlerdeki kurulumlarda eksik oluyor) yüklü olmalı. https://jupyter.org/install adresinden yönergeleri izledim ve 8888 portuna kuruldu.

In [ ]:
# Import necessary packages
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.ndimage import zoom
from skimage import exposure

Gerekli importlar yapıldı, ek olarak random ve ImageGrid kütüphanelerini de ekledim.

In [ ]:
# define most used methods for project

#get images min-max values
def get_image_min_max_val(img):
    return img.min(), img.max()

#get an img and convert to contrast stretched mode
def get_image_contrast_stretched(img,min,max):
    cs=(img - min) / (max - min)# Returns values in [0, 1]
    return (cs * 255).astype(np.uint8)


# get a contrast stretched image to convert equalized image
def get_equalized_image(cs):
    ei= exposure.equalize_hist(cs)# Returns values in [0, 1]
    return (ei * 255).astype(np.uint8)

#get an equalized image to convert gamma corrected image
def get_gamma_corrected(ei):
     gc = np.power(ei / 255.0, gamma) * 255
     return gc.astype(np.uint8)

#get an image name and convert a processable image
def get_raw_image(image_name):
    return plt.imread(os.path.join(img_dir, image_name))

birden çok yerde kullanacağım bazı methodları buraya ekledim.

In [ ]:
# clone the datas, change directory
!git clone https://github.com/hardik0/AI-for-Medicine-Specialization
%cd AI-for-Medicine-Specialization/AI-for-Medical-Diagnosis/

Sizin verdiğiniz resimleri indirme ve klasöre geçme kısmı

In [ ]:
#print inside nih
%ls nih/

# Read csv file containing training datadata
train_df = pd.read_csv("nih/train-small.csv")
# Print first 5 rows
print(f'There are {train_df.shape[0]} rows and {train_df.shape[1]} columns in this data frame')
train_df.head()

train_df.info()

print(f"The total patient ids are {train_df['PatientId'].count()}, from those the unique ids are {train_df['PatientId'].value_counts().shape[0]} ")
columns = train_df.keys()
columns = list(columns)
print(columns)

for column in columns:
    print(f"The class {column} has {train_df[column].sum()} samples")

# Extract numpy values from Image column in data frame
images = train_df['Image'].values

# Extract 9 random images from it
random_images = [np.random.choice(images) for i in range(9)]

# Location of the image dir
img_dir = 'nih/images_small/'

#gamma value
gamma = 1.2

print('Display Random Images')

# Adjust the size of your images
plt.figure(figsize=(20,10))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')

# Adjust subplot parameters to give specified padding
plt.tight_layout()

yine sizin verdiğiniz rastgele 9 resim seçtirip ekrana 3x3 şeklinde bastırma kısmı

In [ ]:
#create a new fig 
new_fig = plt.figure(figsize=(15, 15))
# use imagegrid library for get a grid
grid = ImageGrid(new_fig, 111,
                 nrows_ncols=(3, 3),  # creates 3x3 grid of axes
                 axes_pad=1,  # pad between axes
                 )

#showing images in a 3x3 grid with foreach
for ax, img in zip(grid,random_images):

    raw_image=get_raw_image(img)
    ax.set_title(img)
    ax.imshow(raw_image, cmap='gray')

    #print the values
    print(f"The dimensions of the {img} image are {raw_image.shape[0]} pixels width and {raw_image.shape[1]} pixels height, one single color channel")
    print(f"The maximum {img} image pixel value is {raw_image.max():.4f} and the minimum is {raw_image.min():.4f}")
    print(f"The mean value of the {img} pixels is {raw_image.mean():.4f} and the standard deviation is {raw_image.std():.4f}")

Burada ImageGrid kütüptanesini kullanran 3x3 boyutunda bir grid'e resimleri bastırdım.

In [ ]:
#set figure size
plt.figure(figsize=(20,20))

#foreach images
for i, img in enumerate(random_images):
    
    #set 3x3 grid and auto assign where image display in the grid 
    plt.subplot(3, 3, i + 1)
    #set raw image
    raw_image = get_raw_image(img)

    #draw histograms for all images
    sns.distplot(raw_image.ravel(),
             label=f'Pixel Mean {np.mean(raw_image):.4f} & Standard Deviation {np.std(raw_image):.4f}', kde=False)
    plt.legend(loc='upper center')
    #set image name title 
    plt.title(img)
    plt.xlabel('Pixel Intensity')
    plt.ylabel('# Pixels in Image')

Burada enumerate etmemin sebebi sizin otomatik olarak resmin yerini belirlediğiniz yapıyı bozmamak. For döngülerini de foreach'lere çevirdim.

In [ ]:
#draw all random images and their histograms. original,Contrast-Stretched,contrast-equalized,gamma-corrected
for img in random_images:

    raw_image = get_raw_image(img)

    min_val, max_val = get_image_min_max_val(raw_image)[0],get_image_min_max_val(raw_image)[1]

    contrast_stretched = get_image_contrast_stretched(raw_image,min_val,max_val)
    equalized_image = get_equalized_image(contrast_stretched)
    gamma_corrected = get_gamma_corrected(equalized_image)

    # Plot original image and histogram
    print(img+" to be processed")
    plt.figure(figsize=(40, 20))

    plt.subplot(2, 4, 1)
    plt.imshow(raw_image, cmap='gray')
    plt.title('Original Image ' +img)
    plt.colorbar()
    plt.subplot(2, 4, 5)
    plt.hist(raw_image.ravel(), bins=256, color='gray')
    plt.title("Original Image Histogram "+img)

    # Plot contrast-stretched image and histogram
    plt.subplot(2, 4, 2)
    plt.imshow(contrast_stretched, cmap='gray')
    plt.title('Contrast-Stretched Image '+img)
    plt.colorbar()
    plt.subplot(2, 4, 6)
    plt.hist(contrast_stretched.ravel(), bins=256, color='gray')
    plt.title("Contrast-Stretched Histogram "+img)

    # Plot contrast-equalized image and histogram
    plt.subplot(2, 4, 3)
    plt.imshow(equalized_image, cmap='gray')
    plt.title('Contrast Equalized Image '+img)
    plt.colorbar()
    plt.subplot(2, 4, 7)
    plt.hist(equalized_image.ravel(), bins=256, color='gray')
    plt.title("Equalized Histogram "+img)

    # Plot gamma-corrected image and histogram
    plt.subplot(2, 4, 4)
    plt.imshow(gamma_corrected, cmap='gray')
    plt.title(f'Gamma-Corrected Image (Gamma={1.2}) '+img)
    plt.colorbar()
    plt.subplot(2, 4, 8)
    plt.hist(gamma_corrected.ravel(), bins=256, color='gray')
    plt.title("Gamma-Corrected Histogram "+img)

    plt.tight_layout()
    plt.show()



Burada yine sizin hazırda verdiğiniz kodları foreach ile çoklandırdım. 2 satır 4 sütun olrak filtrelediğimiz resimleri bastırmış olduk

In [ ]:
for img in images:

    raw_image = get_raw_image(img)

    median_filtered = cv2.medianBlur(raw_image, 5)

    plt.figure(figsize=(10, 5))

    # Apply Gaussian Blur
    gaussian_filtered = cv2.GaussianBlur(raw_image, (5, 5), 0)

    # used 1 row and 5 column for showing images
    plt.subplot(1, 2, 1), plt.imshow(median_filtered, cmap='gray'), plt.title("Median Filtered Image "+img)
    plt.subplot(1, 2, 2), plt.imshow(gaussian_filtered, cmap='gray'), plt.title("Gaussian Filtered Image "+img)
    plt.show()

Burada yine sizin hazırda verdiğiniz kodları foreach ile çoklandırdım. Ek olarak 1 satır 2 sütun olarak ekrana bastırdım. Bu kısımlarda subplot() methodunun nasıl çalıştığını kavradım.

In [ ]:
print("We show 3 images side by side, 9 in a row")
for img in random_images:

    #get raw_image
    raw_image = get_raw_image(img)
    #set new figsize
    plt.figure(figsize=(20, 10))
    random_angle=random.randrange(0, 10)
    # get center of raw_images
    center = (raw_image.shape[1] // 2, raw_image.shape[0] // 2)
    #find rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, angle=random_angle, scale=1.0)
    #set ratated_image
    rotated_image = cv2.warpAffine(raw_image, rotation_matrix, (raw_image.shape[1], raw_image.shape[0]))

    # Horizontal Flip
    flipped_horizontal = cv2.flip(raw_image, 1)

    #set a table 1 row 3 column
    plt.subplot(1, 3, 1), plt.imshow(rotated_image, cmap='gray'), plt.title("Rotated "+img+" Image with "+ str(random_angle)+" degree")
    plt.subplot(1, 3, 2), plt.imshow(flipped_horizontal, cmap='gray'), plt.title("Horizontally Flipped Image "+img)

    #fourier transformation
    dft = cv2.dft(np.float32(rotated_image), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shifted = np.fft.fftshift(dft)

    # Create a mask for filtering
    rows, cols = rotated_image.shape
    crow, ccol = rows // 2 , cols // 2
    mask = np.zeros((rows, cols, 2), np.uint8)
    mask[crow-30:crow+30, ccol-30:ccol+30] = 1  # Center low frequencies pass

    # Apply mask and inverse DFT
    fshift = dft_shifted * mask
    f_ishift = np.fft.ifftshift(fshift)
    filtered_image = cv2.idft(f_ishift)
    filtered_image = cv2.magnitude(filtered_image[:, :, 0], filtered_image[:, :, 1])

    # Normalize and display the filtered image
    filtered_image = cv2.normalize(filtered_image, None, 0, 255, cv2.NORM_MINMAX)
    plt.subplot(1, 3, 3),plt.imshow(filtered_image, cmap='gray'),plt.title("Frequency Domain Filtered Image "+img)
    # Display images
    plt.show()

Burada yine sizin hazırda verdiğiniz kodları foreach ile çoklandırdım. 1 satır 3 sütun olarak filtrelediğimiz resimleri bastırdım.

In [ ]:
for img in random_images:
    #get raw image
    raw_image = get_raw_image(img)

    #get min,max
    min_val, max_val = get_image_min_max_val(raw_image)[0],get_image_min_max_val(raw_image)[1]

    #get versions of images
    contrast_stretched = get_image_contrast_stretched(raw_image,min_val,max_val)
    equalized_image = get_equalized_image(contrast_stretched)
    gamma_corrected = get_gamma_corrected(equalized_image)


    # Step 4: Apply sharpening using an unsharp masking kernel
    sharpening_kernel = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
    sharpened_image = np.clip(cv2.filter2D(gamma_corrected, -1, sharpening_kernel), 0, 255).astype(np.uint8)

    # Step 5: Apply bicubic interpolation to upscale the image by a factor of 2
    scale_factor = 2
    resized_image = zoom(sharpened_image, scale_factor, order=3)  # Bicubic interpolation with `order=3`

    # 12,6 figsize, 1row 2 column table
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1), plt.imshow(sharpened_image, cmap='gray'),plt.title("Sharpened Image "+img)
    plt.subplot(1, 2, 2), plt.imshow(resized_image, cmap='gray'),plt.title("Interpolated (Upscaled) Image "+img)


    plt.tight_layout()
    print(img+" to be processed")
    plt.show()

In [ ]:
Burada yine sizin hazırda verdiğiniz kodları foreach ile çoklandırdım. 1 satır 2 sütun olarak resimleri bastırdım.